In [17]:
from wallcheb.qtmlib.circuits.lcu import LCUMultiplexorBox
from guppylang import guppy
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, measure_array, h
from guppylang.std.builtins import array, exit
from typing import Callable


In [18]:
from wallcheb.operators import generate_pytket_hvs_hubbard

u = 1
n_sites = 2
m = 3
product_block_encoding_qpo = generate_pytket_hvs_hubbard(u, n_sites, m)
n_state_qubits = 2*n_sites


/Users/nathanfitzpatric/Github/wall_chebyshev/wallcheb/operators/hubbard_model.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[i,j] = hmat[subset[i], subset[j]]


In [19]:
lcu_box = LCUMultiplexorBox(product_block_encoding_qpo[0], n_state_qubits)
n_prep_qubits = lcu_box.n_prepare_qubits

In [20]:
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes


def build_multiplexor_lcu(ham, n_state_qubits, ind):
    multiplexor_lcu = LCUMultiplexorBox(ham, n_state_qubits)
    

    circ = multiplexor_lcu.get_circuit()
    DecomposeBoxes().apply(circ)
    rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
    rebase.apply(circ)


    qlibs_multiplexor_lcu = guppy.load_pytket(f"qlibs_multiplexor_lcu_{ind}", circ)
    return qlibs_multiplexor_lcu

In [21]:
@guppy.comptime
def guppy_prod_circs() -> array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]],None], comptime(m)]:

    guppy_circuits = [build_multiplexor_lcu(qpo, n_state_qubits, i) for i, qpo in enumerate(product_block_encoding_qpo)]
    return guppy_circuits

In [22]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, discard_array, measure, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from typing import Callable

@guppy
def product_block_encoding(prod_block_encoding: array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]], None], comptime(m)], state_qreg: array[qubit, comptime(n_state_qubits)]) -> None:
    

    for prod_block in prod_block_encoding.copy():

        prep_qreg = array(qubit() for _ in range(comptime(n_prep_qubits)))
        prod_block(prep_qreg, state_qreg)

        outcome = measure_array(prep_qreg)
        # result("c", measure_array(prep_qubits))
        for b in outcome: 
            if b:
                exit("circuit failed",1)


In [26]:
@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    state_qreg = array(qubit() for _ in range(comptime(n_state_qubits)))
    
    for i in range(comptime(n_state_qubits)):
        h(state_qreg[i])

    prods = guppy_prod_circs()

    product_block_encoding(prods, state_qreg)

    result('c',measure_array(state_qreg))

compiled_hugr = guppy.compile(main)


In [27]:
from hugr.qsystem.result import QsysResult
from selene_sim import build, Quest

n_shots = 80000
runner = build(compiled_hugr)
shots = QsysResult(
    runner.run_shots(
        Quest(random_seed=2), n_qubits=lcu_box.n_qubits, n_shots=n_shots
    )
)

print(f'Final state:{shots.register_counts()["c"]}, when initial state is |000>')

shots_counts = shots.register_counts()["c"]
success_prob = sum(shots_counts.values())/ n_shots
print(f"Success probability (from circuits): {success_prob:.4f}")

Final state:Counter({'0110': 14, '1100': 3, '0011': 3, '0101': 2, '1010': 2, '1110': 2, '0000': 2, '1001': 1, '0001': 1, '1101': 1}), when initial state is |000>
Success probability (from circuits): 0.0004
Success probability (from circuits): 0.0004
